In [4]:
import pandas as pd
import os

cache = {}
n_clipped = 0

def clip(t, max_words=500):
    if t not in cache:
        t_words = t.split(" ")
        if len(t_words) > max_words:
            n_missing = int((len(t_words) - max_words) / 2)
            t_words = t_words[n_missing:n_missing + max_words]
            t = " ".join(t_words)
            
        cache[t] = t
    return cache[t]
    
def export_as_formatted(df, stage_name, dir_path):
    entries = []
    queries = []
    responses = []
    
    for i, row in df.iterrows():
        row_query = query_format.format(clip(row["source"]), clip(row["target"]), row["label"])
        row_response = label_creator(row["label"])
        entries.append({
            "prompt": row_query,
            "completion": row_response
        })

    openai_df = pd.DataFrame(entries)
    export_path = os.path.join(dir_path, f"{stage_name}.jsonl")
    openai_df.to_json(export_path,orient='records', lines=True)
    print(f"Exported: {export_path} ({len(openai_df)})")
    return openai_df

data_path = os.path.expanduser("~/desktop/safa/datasets/openai/cm1")
export_dir = os.path.join(data_path, "formatted")

stages = ["train", "test"]
for stage in stages:
    df_path = os.path.join(data_path, f"{stage}.csv")
    df = pd.read_csv(df_path).sample(frac=1)

    query_format = "1. {}\n2. {}\n\n###\n\n"
    label_creator = lambda l: " yes###" if l == 1 else " no###"

    result_df = export_as_formatted(df, stage, export_dir)

Exported: /Users/albertorodriguez/desktop/safa/datasets/openai/cm1/formatted/train.jsonl (67)
Exported: /Users/albertorodriguez/desktop/safa/datasets/openai/cm1/formatted/test.jsonl (23)


In [2]:
result_df["prompt"].map(lambda p: len(p)).max()

7217